In [1]:
## import shit

import folium
import geopandas

from openrouteservice import client, places
from shapely import geometry

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.11) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
## read in SAL

sal = geopandas.read_file('../data/raw/external/SAL_shapefile/SAL_2021_AUST_GDA2020.shp')

## small test before for loop
test = sal.query('SAL_NAME21 == "Wendouree"')
test_shape = test.dissolve(by='SAL_NAME21')
test_shape = test_shape['geometry'][0]

In [3]:
test_json = geometry.mapping(geometry.shape(test_shape))

In [72]:
## appoach one (from pub example)

# bus_station: 587, bus_stop: 588, railway_station: 604, tram_stop: 605, platform: 607, pt_stop_area: 609, pt_station: 610

ors = client.Client(key=api_key)
query = {'request': 'pois',
        'geojson': test_json,
        'filter_category_ids': [604]}
railway_station = ors.places(**query)['features']
railway_station
#railway_station[0]['geometry']['coordinates']

[{'type': 'Feature',
  'geometry': {'type': 'Point', 'coordinates': [143.82002, -37.539683]},
  'properties': {'osm_id': 134721032,
   'osm_type': 2,
   'distance': 0.0,
   'category_ids': {'604': {'category_name': 'station',
     'category_group': 'transport'}},
   'osm_tags': {'name': 'Wendouree'}}}]

In [ ]:
## approach two (from api playground)

import requests

body = {"request":"pois",
        "geometry":{"geojson":test_json},
        "filters":{"category_ids":[604]},
        "limit":10
        }

headers =  {
    'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
    'Authorization': api_key,
    'Content-Type': 'application/json; charset=utf-8'
}

call = requests.post('https://api.openrouteservice.org/pois', json=body, headers=headers)
   
print(call.status_code, call.reason)
print(call.text)

In [4]:
## create the loop
sal_vic = sal.query('STE_NAME21 == "Victoria"')
vic_shapes = sal.dissolve(by='SAL_NAME21')
len(vic_shapes)

15353

Code/Websites used:

For finding Victoria: https://spatialvision.com.au/blog-open-source-spatial-geopandas-part-1/

Model of using ORS: https://openrouteservice.org/example-optimize-pub-crawl-with-ors/

Query categories: https://github.com/GIScience/openpoiservice/blob/master/openpoiservice/server/categories/categories.yml or https://giscience.github.io/openrouteservice/documentation/Places.html

Other model: https://openrouteservice.org/dev/#/api-docs/pois/post